# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [9]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'product page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [11]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [12]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
nvidia/personaplex-7b-v1
Updated
6 days ago
•
43.9k
•
1.35k
moonshotai/Kimi-K2.5
Updated
about 17 hours ago
•
11k
•
854
microsoft/VibeVoice-ASR
Updated
1 day ago
•
76.7k
•
692
Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice
Updated
6 days ago
•
139k
•
687
Tongyi-MAI/Z-Image
Updated
about 5 hours ago
•
552
Browse 2M+ models
Spaces
Running
on
Zero
Featured
926
Qwen3-TTS Demo
🎙
926
Transform text into natural-sounding speech with custom voices
Running
on
Zero
MCP
1.86k
Z Image Turbo
🖼
1.86k
Generate stunning AI images from text descriptions in seco

In [13]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [15]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [16]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 2 relevant links


# Hugging Face Company Brochure

---

## About Hugging Face

Hugging Face is the premier collaboration platform for the machine learning (ML) community, dedicated to building the future of artificial intelligence through openness, collaboration, and ethical AI development. As a central hub for sharing, exploring, and experimenting with open-source ML models, datasets, and applications, it empowers ML engineers, scientists, and enthusiasts worldwide to innovate and learn together.

Founded as a community-driven platform, Hugging Face has rapidly grown to host over 2 million ML models and more than 500,000 datasets spanning modalities including text, images, video, audio, and 3D data. The platform supports seamless collaboration, allowing users to build their portfolios, share projects, and contribute to advancing AI technology.

---

## Platform Highlights

- **Hugging Face Hub:** The go-to space for unlimited public hosting and collaboration on models, datasets, and ML applications.
- **Extensive Library:** Browse and access over 2 million pre-trained models and 500,000 datasets.
- **Spaces:** Interactive AI apps running on the platform featuring demos like natural-sounding text-to-speech, AI image generation, and 3D image manipulation.
- **Open Source Stack:** Tools and libraries designed to accelerate development and research in machine learning.
- **Multimodal Support:** Accommodates all AI formats including text, images, audio, video, and 3D.

---

## Company Culture

Hugging Face nurtures a vibrant and inclusive AI community focused on transparency, education, and ethical AI. The culture encourages collaboration across disciplines, supporting both newcomers and experts in the AI field to connect and contribute meaningfully. Innovation is driven by a passionate science team at the cutting edge of technology, maintaining a strong commitment to building an open and ethical AI future together.

The brand identity reflects its energetic and community-first approach with a bright, approachable style using signature colors like yellow (#FFD21E) and orange (#FF9D00), symbolizing creativity and warmth.

---

## Customers & Community

Hugging Face serves a diverse customer base, including:

- Machine learning engineers and researchers building and sharing models.
- AI startups and enterprises leveraging cutting-edge open-source tools for rapid innovation.
- Educational institutions and students learning ML with accessible resources.
- Enthusiasts and developers experimenting with AI applications across various domains.

The platform’s popularity is evident from trending models developed by top companies such as NVIDIA, Microsoft, and Alibaba, showcasing enterprise-level adoption alongside community contributions.

---

## Careers at Hugging Face

Hugging Face is expanding and actively hiring talent passionate about AI and open-source development. They seek individuals eager to join a fast-growing, dynamic team at the heart of the AI revolution. Available positions range across:

- Machine Learning Research and Engineering
- Software Development
- Community and Developer Relations
- Product Management
- Data Science

Working at Hugging Face means contributing to impactful projects, engaging with a supportive global community, and advancing ethical AI at scale.

Learn more about current job openings and how to join the Hugging Face team on their [Careers page](https://huggingface.co/careers).

---

## Connect & Explore

- **Website:** https://huggingface.co
- **GitHub:** https://github.com/huggingface
- **Twitter:** https://twitter.com/huggingface
- **LinkedIn:** https://linkedin.com/company/huggingface
- **Discord:** https://discord.gg/huggingface

Join Hugging Face to explore AI applications, collaborate on innovative projects, and be part of the AI community building the future.

---

*Hugging Face — The AI community building the future.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [17]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [18]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links


# Hugging Face Brochure

---

## Who We Are

**Hugging Face** is a vibrant AI community and collaboration platform dedicated to building the future of machine learning (ML) and artificial intelligence (AI). Founded in 2016 and based in Paris, France, we are a team of 51-200 passionate professionals specializing in machine learning, natural language processing (NLP), and deep learning. Our mission is to democratize AI through open and ethical development, empowering engineers, scientists, and users worldwide.

---

## What We Offer

- **Hugging Face Hub:** A central, open platform where users can **create, share, and explore** over 2 million cutting-edge ML models, 500,000+ datasets, and 1 million+ AI applications spanning text, image, video, audio, and 3D modalities.

- **Spaces:** Interactive AI-powered apps and demos running on Hugging Face infrastructure, enabling instant exploration of model capabilities (e.g., text-to-speech, AI image generation, 3D camera angle adjustments).

- **Open-Source Stack:** Tools and libraries that accelerate ML experimentation and deployment.

- **Enterprise Solutions:** Tailored offerings for organizations needing advanced AI infrastructure and support.

- **Community:** A rich ecosystem that includes forums, documentation, blogs, and live events fostering collaboration and knowledge-sharing.

---

## Our Culture & Community

At Hugging Face, collaboration and openness fuel our innovation. We believe AI should be transparent, accessible, and built responsibly. Our growing community of machine learning engineers, researchers, and enthusiasts worldwide work together to push the boundaries of AI technology. We foster an environment that values:

- **Ethical AI development:** Commitment to building responsible, fair, and inclusive machine learning systems.

- **Open source:** Encouraging sharing, reuse, and collective improvement of AI models and tools.

- **Continuous learning:** Providing resources and opportunities to build your ML portfolio and stay on the cutting edge.

- **Diversity and Inclusion:** Welcoming talent from all backgrounds to grow our AI community.

---

## Our Customers & Impact

Our platform is trusted by organizations and developers across industries to power applications in:

- Natural language understanding and generation

- Computer vision and image synthesis

- Speech recognition and synthesis

- 3D modeling and augmented reality

Partners and users benefit from access to high-quality open-source models and datasets, enabling rapid innovation without starting from scratch. Hugging Face is at the heart of the AI revolution, fueling breakthroughs that impact research, enterprise AI, and real-world applications alike.

---

## Careers at Hugging Face

We are looking for talented and motivated individuals passionate about machine learning and AI to join our team. Positions in software development, research, product management, and more are regularly available. Join us to help democratize AI and shape its future.

- Enjoy a collaborative and inclusive work environment

- Work alongside top researchers and engineers in ML

- Contribute directly to tools that impact millions globally

To explore current job openings and apply, visit our [Careers page](https://huggingface.co/careers).

---

## Connect With Us

- Website: [huggingface.co](https://huggingface.co)

- LinkedIn: [Hugging Face LinkedIn](https://www.linkedin.com/company/huggingface)

- GitHub, Twitter, Discord: Join our thriving developer and research community on all major platforms.

---

**Hugging Face** — Empowering the next generation of AI through openness, collaboration, and innovation. Join us in building the future of AI together.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>